# SQL Relational Database Demonstration – UK Road Accident Data

**Student:** Anthony Eddei Kwofie  
**Programme:** MSc Artificial Intelligence and Data Science – University of Hull (London Campus)  

---

### Objective
In this task, I demonstrated my ability to query and interpret a relational database using SQL.  
As a data scientist working for the UK government, I explored a provided SQLite database (`accident_data_v1.0.0_2023.db`) containing road accident data.

The main objectives were to:
1. Identify the **oldest driver or rider**.  
2. Count all **vehicles of type 19**.  
3. Retrieve details of **accidents in Kingston upon Hull** using table joins.

I used **Python (sqlite3 + pandas)** to execute SQL queries, view results, and prepare screenshots for my presentation.


### Step 1 – Connecting to the SQLite Database and Inspecting Tables

In this step, I connected to the provided **SQLite database** named `accident_data_v1.0.0_2023.db` that contains the UK road accident dataset used for this assignment.  
I used the **sqlite3** library to establish the connection and **pandas** to query metadata about the database structure.  
By listing the table names, I aimed to confirm that all four expected tables — *accident*, *vehicle*, *casualty*, and *lsoa* — were present before running any SQL queries.


In [9]:
import sqlite3
import pandas as pd

# I established a connection to the provided SQLite database.
conn = sqlite3.connect("accident_data_v1.0.0_2023.db")

# I listed all tables available in the database to confirm successful connection.
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:\n", tables)


Tables in the database:
        name
0  accident
1  casualty
2   vehicle
3      lsoa


### Step 2 – Inspecting the Columns of Each Table

Before running any SQL queries, I examined the column names in each table to understand the database schema.  
This helped me identify the primary keys, foreign keys, and relevant fields needed for my later queries.


In [10]:
# I looped through all table names in the database and used PRAGMA commands
# to display the column names for each table.

for table in tables['name']:
    cols = pd.read_sql_query(f"PRAGMA table_info({table});", conn)
    print(f"\n--- {table.upper()} ---")
    print(cols[['name']])



--- ACCIDENT ---
                                           name
0                                accident_index
1                                 accident_year
2                            accident_reference
3                         location_easting_osgr
4                        location_northing_osgr
5                                     longitude
6                                      latitude
7                                  police_force
8                             accident_severity
9                            number_of_vehicles
10                         number_of_casualties
11                                         date
12                                  day_of_week
13                                         time
14                     local_authority_district
15                 local_authority_ons_district
16                      local_authority_highway
17                             first_road_class
18                            first_road_number
19                    

### Task A – Finding the Age of the Oldest Driver or Rider

I wrote a query to identify the oldest driver or rider recorded in the dataset.  
This helped me test the use of an aggregate function in SQL and understand the upper age range of individuals involved in accidents.


In [11]:
# I used the MAX() aggregate function to find the highest value in the 'age_of_driver' column
# from the 'vehicle' table.

query_a = """
SELECT MAX(age_of_driver) AS oldest_driver
FROM vehicle;
"""

# I executed the query and displayed the result in a pandas DataFrame.
result_a = pd.read_sql_query(query_a, conn)
print(result_a)


   oldest_driver
0            102


### Task B – Counting All Vehicles of Type 19

Next, I wrote a query to count the total number of vehicles classified as type 19 in the database.  
This demonstrated my ability to apply filtering conditions using the `WHERE` clause together with the `COUNT()` aggregate function.


In [12]:
# I used the COUNT() function to calculate the number of records
# in the 'vehicle' table where the 'vehicle_type' equals 19.

query_b = """
SELECT COUNT(*) AS total_vehicle_type_19
FROM vehicle
WHERE vehicle_type = 19;
"""

# I executed the query and displayed the total count of type 19 vehicles.
result_b = pd.read_sql_query(query_b, conn)
print(result_b)


   total_vehicle_type_19
0                  47458


### Task C – Retrieving Accident Details for Kingston upon Hull

Finally, I wrote a more complex SQL query that joined several tables together.  
The aim was to extract the sex of the driver, sex of the casualty, speed limit, and age of the vehicle for all accidents that occurred in **Kingston upon Hull**.  
This demonstrated my ability to perform multi-table joins and filter data using a text condition.


In [13]:
# I joined four related tables: accident, vehicle, casualty, and lsoa.
# The joins connected their primary and foreign keys to combine relevant information.

query_c = """
SELECT 
    v.sex_of_driver,
    c.sex_of_casualty,
    a.speed_limit,
    v.age_of_vehicle,
    l.lsoa01nm AS region
FROM accident a
JOIN vehicle v ON a.accident_index = v.accident_index
JOIN casualty c ON a.accident_index = c.accident_index
JOIN lsoa l ON a.lsoa_of_accident_location = l.lsoa01cd
WHERE l.lsoa01nm LIKE '%Kingston upon Hull%';
"""

# I executed the query and viewed a sample of the results along with the total number of rows.
df = pd.read_sql_query(query_c, conn)
print(df.head())        # display first few rows
print("Total rows:", len(df))


   sex_of_driver  sex_of_casualty  speed_limit  age_of_vehicle  \
0              1                1           30              13   
1              1                1           30               9   
2              3                1           30               6   
3              3                2           30               6   
4              1                1           30              11   

                    region  
0  Kingston upon Hull 028C  
1  Kingston upon Hull 028C  
2  Kingston upon Hull 029A  
3  Kingston upon Hull 029A  
4  Kingston upon Hull 029A  
Total rows: 6387


### Step 7 – Closing the Database Connection

After completing all my SQL queries, I closed the database connection to release system resources and ensure good data management practice.


In [14]:
# I closed the connection to the SQLite database after completing all queries.
conn.close()
